In [ ]:
import sys
import os
from dotenv import load_dotenv

load_dotenv()

sys.path += [
    '/home/ashwin/local/llama-agent-system',
    f'{os.path.expanduser("~/llama-agent-system")}'
]

# Uncomment your path in each list
MP1_CHECKPOINT_DIR = [
    # "/home/ashwin/local/checkpoints/llama3_7b_01_05_2024_torchx_launch_v5-qzn2md/",
    "/home/dalton/llama3_7b_01_05_2024_torchx_launch_v5-qzn2md"
][0]

MP1_TOKENIZER_PATH = [
    # "/home/ashwin/local/checkpoints/llama3_7b_01_05_2024_torchx_launch_v5-qzn2md/cl_toplang_128k",
    "/home/dalton/cl_toplang_128k",
][0]

MP8_CHECKPOINT_DIR = [
    os.path.abspath(os.path.expanduser("~/models.llama3_1.api_70b_full_rlhf3_14042024/"))
][0]

MP8_TOKENIZER_PATH = [
    os.path.abspath(os.path.expanduser("~/models.llama3_1.api_70b_full_rlhf3_14042024/cl_toplang_128k.model"))
][0]

os.chdir('/home/dalton/llama-agent-system/') # TODO: needed to get pwd for examples/ to work

In [ ]:
from llama_models.llama3_1.api import (
    Attachment,
    Message,
)

from llama_toolchain.inference import GeneratorArgs, LlamaModelParallelGenerator
from llama_agentic_system.import (
    AgenticSystem,
    BraveSearchTool,
    CodeInterpreterTool,
    with_safety,
    WolframAlphaTool,
)

args = GeneratorArgs(
    ckpt_dir=MP8_CHECKPOINT_DIR,
    tokenizer_path=MP8_TOKENIZER_PATH,
    model_parallel_size=8,
    max_seq_len=2048,
    max_batch_size=4,
)

generator = LlamaModelParallelGenerator(args)
generator.start() # Loads model into memory

In [ ]:

tools = [
        WolframAlphaTool(os.environ.get("WOLFRAM_ALPHA_API_KEY")),
        BraveSearchTool(os.environ.get("BRAVE_SEARCH_API_KEY")),
        CodeInterpreterTool(),
    ]

agent = AgenticSystem(
        generator=generator,
        tools=tools,
        max_seq_len=2048,
        input_shields=[],
        output_shields=[],
    )

async def ask_about_transcript(question):
    result = await agent.run(
        dialog=[
            Message(
                role="user",
                content=[
                    f"There is a podcast transcript located at examples/transcript_shorter.txt\n{question}",
                    Attachment(
                        filepath="examples/transcript_shorter.txt",
                        mimetype="text/txt",
                    ),
                ],
            )
        ],
        temperature=0.4,
        top_p=0.9,
        max_gen_len=2048,
    )

    print(result['generation'].content)

In [ ]:
await ask_about_transcript("Summarize salient points from the podcast.")

In [ ]:
await ask_about_transcript("Was Emu mentioned?")

In [ ]:
await ask_about_transcript("What are the key developments in the last 6 months in the podcast's subject area?")

In [ ]:
await ask_about_transcript("Imagine these 2 people meet again in 1 year and do another episode. What could be good follow ups to ask as a host?")

In [ ]:
generator.stop() # Release the GPU memory after you're done